In [29]:
import numpy as np
import pandas as pd
from importlib import reload

In [30]:
# define constants
TRACE_FILE_NAME = 'data/APP_DOWNLOAD.pcapng' # replace with your raw trace .pcapng/.pcap file name
TRACE_PACKET_FEATURE_FILE_NAME = 'data/APP_DOWNLOAD_pkt.csv' # replace with your favorite packet feature .csv file name
TRACE_FLOW_FEATURE_FILE_NAME = 'data/APP_DOWNLOAD_flow.csv' # replace with your favorite flow feature .csv file name

In [ ]:
# convert raw trace to readable packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.generate(TRACE_FILE_NAME,TRACE_PACKET_FEATURE_FILE_NAME)

In [ ]:
# read in packet feature csv file and do some initialization
pkt_feature_df = pd.read_csv(TRACE_PACKET_FEATURE_FILE_NAME)
pkt_feature_df['src_addr'] = pkt_feature_df['ip.src'] + ":" + pkt_feature_df['tcp.srcport'].apply(str)
pkt_feature_df['dst_addr'] = pkt_feature_df['ip.dst'] + ":" + pkt_feature_df['tcp.dstport'].apply(str)

In [ ]:
# view the shape of the dataset: (number of records, number of features)
pkt_feature_df.shape

In [ ]:
# view the data types for each feature
pkt_feature_df.dtypes

In [ ]:
# view the statistical features of each numerical feature
pkt_feature_df.describe()

In [ ]:
# view the first 5 records
pkt_feature_df.head()

In [ ]:
# generate flow features from packet features
from python import flow_feature
reload(flow_feature)
flow_feature_df = flow_feature.generate(pkt_feature_df)
flow_feature_df.to_csv(TRACE_FLOW_FEATURE_FILE_NAME)